In [1]:
import numpy as np
import scipy as sp
import scipy.linalg
import scipy.stats
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
from sklearn.metrics import r2_score
import pickle
import time
from scipy.optimize import fsolve
from scipy.optimize import least_squares
from scipy.optimize import minimize
from scipy import sparse
import os.path
from scipy.interpolate import splrep, splev
import pandas as pd
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
import sys
sys.path.append('../..')
from CIMatrixLib.src.util import *
from CIMatrixLib.src.TreatPattern import *
from CIMatrixLib.src.algorithms.RobustSyntheticControl import *
import CIMatrixLib.src.readData as readData
import importlib

sns.set_style("whitegrid")

## Cleaned Notebook for Semi-Synthetic Experiments

In this notebook, we cleaned the code in main.ipynb and used the causaltensor package to reproduce the Tabel 1 results in https://arxiv.org/pdf/2106.02780.pdf 

In [2]:
pip install causaltensor==0.1.7

Note: you may need to restart the kernel to use updated packages.


## Generate the low-rank matrix $M_0$

In [3]:
def synthetic_M0(n1=50, n2=50, mean_M = 1, r = 10, gamma_shape = 1, gamma_scale = 2, type = 'Gamma'): 
    '''
        generate a random rank-r non-negative (n1 x n2) matrix with mean(M) = mean_M
    '''
    if (type == 'Gamma'):
        U = np.random.gamma(shape = gamma_shape, scale = gamma_scale, size = (n1, r))
        V = np.random.gamma(shape = gamma_shape, scale = gamma_scale, size = (n2, r))
        M0 = U.dot(V.T)
        M0 = M0 / np.mean(M0) * mean_M
    else:
        if (type == 'Gaussian'):
            U = np.random.normal(loc=0, scale = 1, size = (n1, r))
            V = np.random.normal(loc = 0, scale = 1, size = (n2, r))
            M0 = mean_M * U.dot(V.T)
    return M0

## Generate treatment patterns $Z$

In [4]:
def generate_Z(pattern_tuple = ['adaptive'], M0 = 0):
    '''
        generate the binary matrix Z for different patterns 
    '''
    while (True):
        if (pattern_tuple[0] == 'adaptive'):
            a = pattern_tuple[1][0]
            b = pattern_tuple[1][1]
            Z = adpative_treatment_pattern(a, b, M0)
    
        if (pattern_tuple[0] == 'iid'):
            p_treat = np.random.rand()*0.5
            Z = np.random.rand(n1, n2) <= p_treat

        if (pattern_tuple[0] == 'block'):
            m2 = pattern_tuple[1][1]
            Z, treat_units = simultaneous_adoption(pattern_tuple[1][0], m2, M0)
        
        if (pattern_tuple[0] == 'stagger'):
            m2 = pattern_tuple[1][1]
            Z = stagger_adoption(pattern_tuple[1][0], m2, M0)

        ## if some row or some column is all treated; or Z=0; generate Z again  
        if (np.sum(np.sum(1-Z, axis=0) == 0) > 0 or np.sum(np.sum(1-Z, axis=1) == 0) > 0 or np.sum(Z)==0): 
            if (pattern_tuple[0] == 'adaptive'):
                return Z, 'fail'
            continue
        break
    if (pattern_tuple[0] == 'block'):
        return Z, treat_units
    if (pattern_tuple[0] == 'adaptive'):
        return Z, 'success'
    return Z

In [5]:
from causaltensor.cauest import DID
from causaltensor.cauest import DC_PR_with_suggested_rank
from causaltensor.cauest import SDID
from causaltensor.cauest import MC_NNM_with_suggested_rank

In [55]:
def run_algo(algo_list, O, Z, suggest_r=-1, treat_units = [], tau_star = 0, m2 = 0, M0 = 0):
    results = {}
    for (index, algo) in enumerate(algo_list):
        if (algo == 'De-biased Convex'):
            M, tau, std = DC_PR_with_suggested_rank(O, Z, suggest_r=suggest_r)
            
        if (algo == 'MC-NNM'):
            M_b, a, b, tau = MC_NNM_with_suggested_rank(O, 1-Z, suggest_r=suggest_r)
            one_row = np.ones((1, M_b.shape[1]))
            one_col = np.ones((M_b.shape[0], 1))
            M = M_b + a.dot(one_row) + one_col.dot(b.T)

        if (algo == 'robust_synthetic_control'):  
            if (treat_units == []):
                M, tau = stagger_pattern_RSC(O, Z, suggest_r = suggest_r)
            else:
                M, tau = synthetic_control(O, suggest_r=suggest_r, treat_units=treat_units, starting_time=m2)
            #M_1, tau_1 = synthetic_control(O, suggest_r=1, treat_units=treat_units, starting_time=m2) #make the results more robust
            #if (abs(tau_1-tau_star) < abs(tau - tau_star)):
            #    M = M_1
            #    tau = tau_1
        
        if (algo == 'trivial'):
            tau = np.sum(O*Z)/np.sum(Z) - np.sum(O*(1-Z))/np.sum(1-Z)
            M = O - Z * tau

        if (algo == 'OLS'):
            M, tau = DID(O, Z, tau_star = tau_star)

        if (algo == 'ideal'):
            M = M0
            tau = np.sum((O-M0)*Z) / np.sum(Z)

        if (algo == 'SDID'):
            tau = SDID(O, Z)
            M = O - Z * tau

        results[algo] = (M, tau)
    return results

## Semi-synthetic experiments on Sales data

In [56]:

M0 = readData.read_data('sales')
s = np.linalg.svd(M0, full_matrices=False, compute_uv=False)
#print(s)
suggest_r = 35

def sales_experiment_performance_run_results(num_experiment=1, pattern = 'adaptive', suggest_r = 1, tau_star_o = 0):
    samples = np.zeros(num_experiment)
    t1 = time.time()

    algo_list = ['De-biased Convex', 'MC-NNM', 'OLS']

    datas = np.zeros((num_experiment, len(algo_list)))

    (n1, n2) = M0.shape

    for T in range(num_experiment):
        if (T % 100 == 0):
            print(time.time() - t1)
            print('experiment ', T)
        
        if (pattern == 'adaptive'):
            while True:
                a = np.random.randint(20)+5
                b = np.random.randint(20)+5
                Z, info = generate_Z(pattern_tuple = ['adaptive', (a, b)], M0=M0)
                if (info == 'fail'):
                    continue
                break
        print('***sparsity****', np.sum(Z) / np.size(Z))


        def test():
            delta = np.random.normal(loc = 0, scale = tau_star_o / 2, size = (n1, 1)) * np.ones((n1, n2))
            d1 = np.sum(Z * delta) / np.sum(Z)
            delta = delta - d1
            tau_star = tau_star_o + d1

            O = M0 + Z*delta + tau_star * Z   

            results = run_algo(algo_list, O, Z, suggest_r = suggest_r, tau_star = tau_star, M0 = M0)
            
            error_metric = {}
            for algo in algo_list:
                (M, tau) = results[algo]
                error_metric[algo] = metric_compute(M, tau, M0, tau_star, Z, ['tau_diff'])['tau_diff']
            return error_metric

        error_metric = test()
        print(error_metric)
        for index, algo in enumerate(algo_list):
                datas[T, index] = error_metric[algo]
        print('experiment {}, time elapses '.format(T), time.time() - t1)
    datas = pd.DataFrame(datas, columns = algo_list)
    return datas


tau_star_o = np.mean(M0)/5
datas = sales_experiment_performance_run_results(num_experiment = 100, pattern = 'adaptive', suggest_r = suggest_r, tau_star_o = tau_star_o)

5.0067901611328125e-06
experiment  0
***sparsity**** 0.45883742718644344
{'De-biased Convex': 119.36136456306122, 'MC-NNM': -44.335364818712605, 'OLS': -345.9704710994897}
experiment 0, time elapses  61.60319685935974
***sparsity**** 0.18982443276711883
{'De-biased Convex': 44.26054155303564, 'MC-NNM': -336.8623732580959, 'OLS': -254.53508979693288}
experiment 1, time elapses  118.2247622013092
***sparsity**** 0.3570410200008147
{'De-biased Convex': 87.42465922528527, 'MC-NNM': 172.98232105680336, 'OLS': 186.30593442684176}
experiment 2, time elapses  178.24026799201965
***sparsity**** 0.27846348120086356
{'De-biased Convex': -0.691351964572732, 'MC-NNM': -129.6770810229932, 'OLS': -195.91323803115938}
experiment 3, time elapses  237.66384410858154
***sparsity**** 0.1684386329382052
{'De-biased Convex': -21.105153437695208, 'MC-NNM': -32.12182763489545, 'OLS': -223.7774878992218}
experiment 4, time elapses  293.0288188457489
***sparsity**** 0.15479245590451748
{'De-biased Convex': -57.

In [57]:
(np.abs(datas) / tau_star_o).describe()

,De-biased Convex,MC-NNM,OLS
count,100.000000,100.000000,100.000000
mean,0.024979,0.161725,0.210507
std,0.018946,0.104833,0.193478
min,0.000284,0.002017,0.000208
25%,0.007024,0.070187,0.068087
50%,0.022851,0.139532,0.143642
75%,0.038415,0.249639,0.313022
max,0.072139,0.400072,0.727699


## Semi-synthetic experiments on Tobacco data

In [63]:
def tobacco_experiment_performance_run_results(num_experiment=1, sigma = 0.1, pattern = 'block', suggest_r = 10, row_specific=False):
    t1 = time.time()

    algo_list = ['De-biased Convex', 'SDID', 'MC-NNM', 'robust_synthetic_control', 'OLS']

    datas = np.zeros((num_experiment, len(algo_list)))

    (n1, n2) = M0.shape

    for T in range(num_experiment):
        if (T % 100 == 0):
            print(time.time() - t1)
            print('experiment ', T)

        ## generating stagger pattern Z
        if (pattern == 'stagger'):
            m1 = np.random.randint(low=1, high=n1)
            m2 = np.random.randint(low=1, high=n2)
            Z = generate_Z(pattern_tuple=['stagger', (m1, m2)], M0=M0)
            treat_units = []

        if (pattern == 'block'):
            #m1 = np.random.randint(low=1, high=int(n1/3))
            #m2 = np.random.randint(low=int(n2/2), high=n2)
            m1 = np.random.randint(low=1, high=5)
            m2 = 18
            Z, treat_units = generate_Z(pattern_tuple=['block', (m1, m2)], M0=M0)
            print('***sparsity****', np.sum(Z) / np.size(Z))

        def test():

            #M0, M1, E = synthetic_intervention_pattern()
            delta = np.random.normal(loc = 0, scale = tau_star_o / 2, size = (n1, 1)) * np.ones((n1, n2))
            #print(delta)
            d1 = np.sum(Z * delta) / np.sum(Z)
            delta = delta - d1
            tau_star = tau_star_o + d1

            O = M0 + Z*delta + tau_star * Z 
            
            results = run_algo(algo_list, O, Z, suggest_r = suggest_r, treat_units=treat_units, tau_star = tau_star, m2 = m2, M0 = M0)


            error_metric = {}
            for algo in algo_list:
                (M, tau) = results[algo]
                error_metric[algo] = metric_compute(M, tau, M0, tau_star, Z, ['tau_diff'])['tau_diff']
            
            return error_metric

        error_metric = test()
        print(error_metric)
        for index, algo in enumerate(algo_list):
                datas[T, index] = error_metric[algo]
        print('experiment {}, time elapses '.format(T), time.time() - t1)
    datas = pd.DataFrame(datas, columns = algo_list)
    return datas

M0 = readData.read_data('tobacco')
suggest_r = 5
tau_star_o = np.mean(M0) / 5
datas = tobacco_experiment_performance_run_results(num_experiment = 100, pattern = 'stagger', suggest_r = suggest_r)

3.0994415283203125e-06
experiment  0
{'De-biased Convex': -1.4086585070538717, 'SDID': -11.583023929340628, 'MC-NNM': -8.476170089953191, 'robust_synthetic_control': -9.883489101804308, 'OLS': -7.265871878499794}
experiment 0, time elapses  10.003129243850708
{'De-biased Convex': -0.7386708703147207, 'SDID': -4.950631227388357, 'MC-NNM': -7.817709697042961, 'robust_synthetic_control': 0.20256873318528434, 'OLS': -12.35075877693222}
experiment 1, time elapses  19.976001024246216
{'De-biased Convex': 6.000168455199422, 'SDID': -0.43612063738395435, 'MC-NNM': 2.2231631419361833, 'robust_synthetic_control': 2.064237405923109, 'OLS': 0.09596083883039697}
experiment 2, time elapses  26.132344245910645
{'De-biased Convex': -1.5662981363313797, 'SDID': -2.0012248618727853, 'MC-NNM': -0.6167917208729392, 'robust_synthetic_control': 0.16320663631941912, 'OLS': -1.324566132238676}
experiment 3, time elapses  31.638011932373047
{'De-biased Convex': -3.3260405743948525, 'SDID': 7.572842385867222, '

In [64]:
(np.abs(datas) / tau_star_o).describe()

,De-biased Convex,SDID,MC-NNM,robust_synthetic_control,OLS
count,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.101179,0.178583,0.167516,0.216767,0.217179
std,0.084983,0.135128,0.149479,0.288645,0.168821
min,0.000903,0.001092,0.001182,0.004133,0.001887
25%,0.037007,0.081717,0.057550,0.061223,0.075258
50%,0.077180,0.147395,0.132094,0.139841,0.174847
75%,0.148142,0.236364,0.225003,0.265388,0.305811
max,0.461141,0.630143,0.821092,2.187002,0.872263


In [61]:
M0 = readData.read_data('tobacco')
suggest_r = 5
tau_star_o = np.mean(M0) / 5
datas = tobacco_experiment_performance_run_results(num_experiment = 100, pattern = 'block', suggest_r = suggest_r)

5.0067901611328125e-06
experiment  0
***sparsity**** 0.011035653650254669
{'De-biased Convex': -0.7961720621223733, 'SDID': -17.211353518873, 'robust_synthetic_control': -4.402694792626679, 'OLS': -14.181068604712726}
experiment 0, time elapses  0.19588613510131836
***sparsity**** 0.011035653650254669


/var/folders/84/sq9gqqh952qfzb3rpnrkcjqr0000gn/T/ipykernel_2056/2130466035.py:14: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if (treat_units == []):


{'De-biased Convex': -0.6430926473850125, 'SDID': 3.5849302267763203, 'robust_synthetic_control': -3.8299516812690566, 'OLS': 22.31919016443137}
experiment 1, time elapses  0.3824329376220703
***sparsity**** 0.03310696095076401


/var/folders/84/sq9gqqh952qfzb3rpnrkcjqr0000gn/T/ipykernel_2056/2130466035.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if (treat_units == []):


{'De-biased Convex': 0.38991622437369244, 'SDID': -3.9442202546208556, 'robust_synthetic_control': -16.862091410903524, 'OLS': -14.058980108220021}
experiment 2, time elapses  0.5848729610443115
***sparsity**** 0.03310696095076401
{'De-biased Convex': 11.79489257312921, 'SDID': 6.405807808580569, 'robust_synthetic_control': 11.716210583637334, 'OLS': 0.023928976695167137}
experiment 3, time elapses  0.7609341144561768
***sparsity**** 0.044142614601018676
{'De-biased Convex': 4.04521953370012, 'SDID': -0.13244318234424668, 'robust_synthetic_control': 3.595779428424173, 'OLS': 0.6524752524776787}
experiment 4, time elapses  0.938647985458374
***sparsity**** 0.022071307300509338
{'De-biased Convex': 3.3512391406441573, 'SDID': -10.474822708677161, 'robust_synthetic_control': -33.306703664252254, 'OLS': -37.700317911904136}
experiment 5, time elapses  1.1440010070800781
***sparsity**** 0.03310696095076401
{'De-biased Convex': 0.036322454943253035, 'SDID': 3.8393153335428423, 'robust_synthe

In [62]:
(np.abs(datas) / tau_star_o).describe()

,De-biased Convex,SDID,robust_synthetic_control,OLS
count,100.000000,100.000000,100.000000,100.000000
mean,0.124280,0.215341,0.325753,0.370642
std,0.111113,0.180700,0.276591,0.324225
min,0.000454,0.000337,0.006197,0.001001
25%,0.038305,0.077565,0.145288,0.117805
50%,0.091922,0.170339,0.271645,0.307461
75%,0.184942,0.318542,0.414284,0.527908
max,0.553707,0.783723,1.541961,1.577221
